In [29]:
import os
import cv2

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from PIL import Image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [19]:
def plot_images(images):
    
    n_images = len(images)
    fig, ax = plt.subplots(1, n_images)
    
    for i, img in enumerate(images):
        ax[i].imshow(img)
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        
    plt.tight_layout()
    plt.show()
    
    

    

In [132]:
PATH = '/home/weders/scratch/scratch/scannetter/arkit/raw/Validation'
SCENE = '42897688'

In [133]:
scene_path = os.path.join(PATH, SCENE)
image_path = os.path.join(scene_path, 'vga_wide')
depth_path = os.path.join(scene_path, 'highres_depth')
intrinsics_path = os.path.join(scene_path, 'vga_wide_intrinsics')
pose_path = os.path.join(scene_path, 'lowres_wide.traj')

In [134]:
def axis_angle_to_mat(angles):
    matrix, jac = cv2.Rodrigues(angles)
    return matrix

def load_intrinsics(file):
    # as define here https://github.com/apple/ARKitScenes/blob/951af73d20406acf608061c16774f770c61b1405/threedod/benchmark_scripts/utils/tenFpsDataLoader.py#L46
    w, h, fx, fy, hw, hh = np.loadtxt(file)
    return np.asarray([[fx, 0, hw], [0, fy, hh], [0, 0, 1]])


def load_poses(file):
    trajectory = {}
    with open(file, 'r') as f:
        for line in f:
            elements = line.rstrip().split(' ')
            
            timestamp = round(float(elements[0]), 3)
            rotation = axis_angle_to_mat(np.asarray([float(e) for e in elements[1:4]]))
            translation = np.asarray([float(e) for e in elements[4:]])
            
            pose = np.eye(4)
            pose[:3, :3] = rotation
            pose[:3, -1] = translation
            
            trajectory[timestamp] = {'rotation': rotation, 'translation': translation, 'pose': pose}
    return trajectory
    
poses = load_poses(pose_path)
pose_timestamps = list(poses)


def get_closest_timestamp(timestamp, pose_timestamps):
    closest_timestamp = np.infty
    closest_delta = np.infty
    
    for ts in pose_timestamps:
        delta = abs(timestamp - ts)
        if delta < closest_delta:
            closest_timestamp = ts
            closest_delta = delta
    return closest_timestamp
        
    

In [135]:
from tqdm import tqdm

In [136]:
tsdf = o3d.pipelines.integration.ScalableTSDFVolume(sdf_trunc=0.06, 
                                                    voxel_length=0.02, 
                                                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

intrinsics_loaded = False

image_timestamps = [float(f.split('_')[-1].replace('.png', '')) for f in os.listdir(image_path)]

files = os.listdir(depth_path)
pcds = None

for file in  tqdm(files, total=len(files)):
    
    depth_timestamp = float(file.split('_')[-1].replace('.png', ''))

    image_file = get_closest_timestamp(depth_timestamp, image_timestamps)    
    image_file = f"{SCENE}_{image_file:.3f}.png"
    
    if not os.path.exists(os.path.join(image_path, image_file)):
        print(image_file, 'not found')
        continue
    
    if not intrinsics_loaded:
        intrinsics = load_intrinsics(os.path.join(intrinsics_path, image_file.replace('png', 'pincam')))
        intrinsics = o3d.camera.PinholeCameraIntrinsic(width=w, height=h, fx=intrinsics[0, 0], fy=intrinsics[1, 1], cx=intrinsics[0, 2], cy=intrinsics[1, 2])
        intrinsics_loaded = False
        
    image = np.asarray(Image.open(os.path.join(image_path, image_file))).astype(np.uint8)
    depth = np.asarray(Image.open(os.path.join(depth_path, file))).astype(np.float32) / 1000.

    h, w, _ = image.shape
    
    depth = cv2.resize(depth, (w, h))
    
    timestamp = float(file.replace('.png', '').split('_')[-1])
    c_ts = get_closest_timestamp(timestamp, pose_timestamps)
    
    if c_ts == np.infty:
        print('No matching pose')
        continue
    
    pose = poses[c_ts]['pose']
#     plot_images([image, depth])
    
    image = o3d.geometry.Image(image)
    depth = o3d.geometry.Image(depth)
    
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(image, 
                                                              depth,
                                                              depth_scale=1.0,
                                                              depth_trunc=3.,
                                                              convert_rgb_to_intensity=False)
    
    if pcds is None:
        
        pcds = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsics, pose).voxel_down_sample(0.04)
    
    else:
        pcds = pcds + o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsics, pose)
        pcds = pcds.voxel_down_sample(0.01)
    
    
    
    
    
o3d.io.write_point_cloud(f'{SCENE}.ply', pcds)

100%|████████████████████████████████████| 540/540 [05:23<00:00,  1.67it/s]


True